<img src="https://i.imgur.com/6U6q5jQ.png"/>

# Formatting Dates in Python

It is very common to find dates (some combination of year, month, day of week and time) in data that is collected in real time (and other that organize event information.

Let's see a data frame that comes with dates from an API.

In [ ]:
#!pip install sodapy

In [ ]:
import pandas as pd
from sodapy import Socrata

client = Socrata("data.seattle.gov", None)

results = client.get("kzjm-xkqj", limit=2000)

# Convert to pandas DataFrame
calls911 = pd.DataFrame.from_records(results)

Let's check the data types:

In [ ]:
calls911.info()

Let's get rid of some columns:

In [ ]:
calls911=calls911.iloc[:,:7]

Let's check the column _datetime_:

In [ ]:
calls911.datetime.head()

In [ ]:
# verify data type
type(calls911.datetime[0])


The date and time information is not useful at this time, that is, the information it offers is of limited use, as it is just a string. 

Let's make it useful:

In [ ]:
calls911.datetime=pd.to_datetime(calls911.datetime)
calls911.info()

In [ ]:
calls911

Once you have this data type, you can retrieve important information:

In [ ]:
calls911['date']=calls911.datetime.dt.date
calls911['year']=calls911.datetime.dt.year
calls911['month']=calls911.datetime.dt.month_name()
calls911['weekday']=calls911.datetime.dt.day_name()
calls911['hour']=calls911.datetime.dt.hour

In [ ]:
calls911.head()

Let's create a new column with what we have. In this case, a boolean where you tell if it is night time (after 8 pm before 6 am):

In [ ]:
calls911['nightTime']=((calls911['hour']<=6) | (calls911['hour']>=20))

Let's save what we have:

In [ ]:
calls911

What about data that comes in Spanish?

In [ ]:
#!pip install bs4

In [ ]:
link="https://es.wikipedia.org/wiki/Pandemia_de_COVID-19"

import pandas as pd

covid=pd.read_html(link, flavor="bs4", attrs={"class":"wikitable sortable"})

Let me keep the second df:

In [ ]:
covidDF=covid[1].copy()
covidDF

Notices the presence of some non-English punctuation:

In [ ]:
covidDF.columns

Let's get rid of those:

In [ ]:
#!pip install unidecode

In [ ]:
import unidecode as ud

[ud.unidecode(c) for c in covidDF.columns]

In [ ]:
#or
import re

[re.sub('\\s','',ud.unidecode(c)) for c in covidDF.columns]

In [ ]:
#then
covidDF.columns=[re.sub('\\s','',ud.unidecode(c)) for c in covidDF.columns]

Let's  focus on the _Fechadelanalisis_:

In [ ]:
# use " a " to split:
covidDF.Fechadelanalisis.str.split(" a ",expand=True)

In [ ]:
# create the two columns

covidDF[["fecha1","fecha2"]]=covidDF.Fechadelanalisis.str.split(" a ",expand=True)
covidDF

Let's format on of those columns:

In [ ]:
covidDF.fecha1

In [ ]:
covidDF.loc[8,'fecha1']='1 de noviembre de 2020'

In [ ]:
# let's split again:

covidDF.fecha1.str.split(" de ",expand=True)

I could create three new columns:

In [ ]:
covidDF[['fecha1_dia','fecha1_mes','fecha1_anho']]=covidDF.fecha1.str.split(" de ",expand=True)
covidDF[['fecha1_dia','fecha1_mes','fecha1_anho']]

We should use the month number instead of name. Let's prepare a dict of changes:

In [ ]:
monthName=('enero','febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre')
changes={name:number for name,number in zip(monthName,range(1,len(monthName)+1))}
changes

In [ ]:
covidDF.fecha1_mes.replace(changes,inplace=True)

Now we have:

In [ ]:
covidDF[['fecha1_dia','fecha1_mes','fecha1_anho']]

We will use those columns to create a date:

In [ ]:
pd.to_datetime(dict(year=covidDF.fecha1_anho, month=covidDF.fecha1_mes, day=covidDF.fecha1_dia))